In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
from torch.utils.data import Dataset
import pickle
import datetime
import pandas as pd

In [ ]:
if(torch.cuda.is_available()):
    dev = torch.device("cuda")
else:
    dev = torch.device("cpu")
print(dev)

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import csv
training_data = []
with open('/content/drive/MyDrive/database_paris_marseille copy.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        row[0] = float(row[0])
        if row[0]>90:
            row[0] = row[0]-360
        training_data.append([float(item) for item in row])

with open("/content/drive/MyDrive/objet_wind_data_2020.pickle", "rb") as f:
    data= pickle.load(f)
    wind_data = data['data']


In [ ]:
angles = np.arctan2(wind_data[:,:,:,:,1], wind_data[:,:,:,:,0])
angles = np.round(((angles*360/(2*np.pi))/45))+4
directions = [np.where(angles == i , 1, 0) for i in range(1, 9)]
for i in range(8):
    directions[i] = np.sum(directions[i], axis = 1)
directions = np.array(directions)
direction_data= np.where(directions > 0 , 1, 0)

NameError: name 'np' is not defined

In [ ]:
data_std = 1

def grid_time(t):
    t = t
    t = t/3600
    t = t + 1 + 4
    t = t - t%6
    t = t - 1
    return (t+1)/6

def grid_lat(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5
    l = l/2
    l = l +90
    l = l*2

    return (l/5)%73

def grid_long(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5

    return (l/5)%144
def int_array(arr):
    return [int(i) for i in arr]

def grid_array(arr):
    return int_array([grid_long(arr[0]),grid_lat(arr[1]),grid_time(arr[2]),int(arr[3]),int(arr[4])])

def get_winds_at_t(data,time,dtime):
    tensor = torch.tensor([data[time:time+dtime,:,:,:]], dtype=torch.float32).permute(1,0,5,2,3,4)
    return tensor


def input_label(training_data,dtime,i, goal_coord,data_winds, data_dirs):
    array = grid_array(training_data[i])
    time = array[2]
    long = array[0]
    lat = array[1]
    alt = array[3]
    dirs = data_dirs[:,time:time+dtime,:,:]
    dirs = np.transpose(dirs,(1,0,2,3))
    pos = np.zeros((dtime,1,dirs.shape[2],dirs.shape[3]))
    pos[0,0,int(grid_long(goal_coord[0])),int(grid_lat(goal_coord[1]))] = -data_std*1.3
    pos[0,0,long,lat] = data_std*1.3
    input = np.concatenate((dirs,pos),axis=1)
    action = np.arctan2(data_winds[time,alt,long,lat,1],data_winds[time,alt,long,lat,0])
    action = int(np.round(((action+np.pi)*360/(2*np.pi))/45)%8)

    return (input,action)


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, entries, dtime, goal_coord, wind, direction_data, transform=None, target_transform=None):
        self.entries = entries
        self.transform = transform
        self.target_transform = target_transform
        self.dtime = dtime
        self.goal_coord = goal_coord
        self.wind_data = wind_data
        self.direction_data = direction_data


    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        inp,lab = input_label(self.entries,self.dtime,idx, self.goal_coord,self.wind_data, self.direction_data)
        inp = np.pad(inp,((0,0),(0,0),(2,2),(0,0)),'wrap')
        return inp, lab

NameError: name 'Dataset' is not defined

In [ ]:
coords_destination = [5.3698, 43.2965]
trainset = CustomDataset(training_data,1,coords_destination,wind_data,direction_data)
batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)


In [ ]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
print(a.shape)
print(b)


torch.Size([32, 1, 9, 148, 73])
tensor([6, 1, 2, 4, 5, 2, 2, 0, 4, 4, 2, 4, 4, 4, 1, 3, 3, 5, 3, 2, 4, 0, 3, 5,
        1, 4, 6, 2, 3, 3, 5, 3])


In [ ]:
class Net(nn.Module):
    def __init__(self , print_shape=True):
        self.print_shape = print_shape
        super(Net, self).__init__()
        self.layersCNN = nn.ModuleList( [
        nn.Conv2d(9, 32, (5,5)),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)),
        nn.Dropout(0.3),
        nn.Conv2d(32, 32, (3,3), padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Conv2d(32, 64, (3,3), padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)),
        nn.Dropout(0.3),
        nn.Conv2d(64, 64, (3,3), padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Conv2d(64, 128, (3,3), padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2), stride=(2,1)),
        nn.Dropout(0.3),
        nn.Conv2d(128, 128, (3,3), padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Conv2d(128, 256, (3,3), padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)),
        nn.Dropout(0.3),
        nn.Conv2d(256, 256, (3,3), padding=0),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Conv2d(256, 512, (3,3), padding=0),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)),
        nn.Dropout(0.3)])
        self.layers = nn.ModuleList([
        nn.Linear(2048, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(1024, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 8)])

        for l in self.layers:
            if isinstance(l, (nn.BatchNorm2d, nn.BatchNorm1d)):
                l.weight.data.fill_(1)
                l.bias.data.zero_()
            elif isinstance(l, nn.Conv2d):
                n = l.kernel_size[0] * l.kernel_size[1] * l.out_channels
                l.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(l, nn.Linear):
                nn.init.xavier_normal(l.weight)

    def forward(self, x):
        x = x[:,0,:,:,:]
        for l in self.layersCNN:
            x = l(x)
            if isinstance(l , nn.Dropout) and self.print_shape:
                print(x.shape)
        x = torch.flatten(x, 1)
        for l in self.layers:
            x = l(x)
            if isinstance(l , nn.Dropout) and self.print_shape:
                print(x.shape)
        return x

NameError: name 'nn' is not defined

In [ ]:
net = Net(print_shape=False).to(dev)
criterion = nn.CrossEntropyLoss().to(dev)


<ipython-input-63-c94f4e09f9ef>:70: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(l.weight)


In [ ]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
r = net(a)
print(b)


tensor([4, 6, 3, 5, 1, 5, 3, 2, 4, 0, 2, 4, 4, 4, 7, 5, 3, 4, 3, 4, 1, 2, 0, 0,
        5, 3, 1, 1, 6, 3, 3, 4])


0

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.00001)
# exp_lr_scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [ ]:
for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    # exp_lr_scheduler.step()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.float().to(dev)
        labels = labels.to(dev)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 1.415
[1,   400] loss: 1.429
[1,   600] loss: 1.439
[1,   800] loss: 1.433
[1,  1000] loss: 1.422
[1,  1200] loss: 1.414
[1,  1400] loss: 1.429
[1,  1600] loss: 1.416
[1,  1800] loss: 1.428
[1,  2000] loss: 1.423
[1,  2200] loss: 1.422
[1,  2400] loss: 1.421
[1,  2600] loss: 1.413
[1,  2800] loss: 1.426
[1,  3000] loss: 1.425
[1,  3200] loss: 1.425
[1,  3400] loss: 1.430
[1,  3600] loss: 1.409
[1,  3800] loss: 1.440
[1,  4000] loss: 1.435
[1,  4200] loss: 1.425
[1,  4400] loss: 1.415
[1,  4600] loss: 1.421
[1,  4800] loss: 1.407
[1,  5000] loss: 1.420
[1,  5200] loss: 1.419
[1,  5400] loss: 1.401
[1,  5600] loss: 1.423
[1,  5800] loss: 1.422
[1,  6000] loss: 1.406
[1,  6200] loss: 1.424
[1,  6400] loss: 1.407
[1,  6600] loss: 1.413
[1,  6800] loss: 1.426
[1,  7000] loss: 1.416
[1,  7200] loss: 1.415
[1,  7400] loss: 1.415
[1,  7600] loss: 1.417
[1,  7800] loss: 1.406
[1,  8000] loss: 1.406
[1,  8200] loss: 1.409
[1,  8400] loss: 1.397
[1,  8600] loss: 1.412
[1,  8800] 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 61 %
